In [25]:
import plotly.graph_objects as go
import pandas as pd
import math
colors = {}
colors['Zlnk product based'] = '#1f77b4'
colors['Zlnk local product based'] = '#000000'
colors['Fixed-point product based'] = '#e377c2'
colors['Fixed-point local product based'] = '#9467bd'
colors['Zlnk fam based - explicit'] = '#d62728'
colors['Zlnk fam based - symbolic'] = '#2ca02c'
colors['Zlnk fam based - explicit local'] = '#555555'
colors['Zlnk fam based - symbolic local'] = '#aaaaaa'
colors['Incremental pre-solve'] = '#8c564b'
colors['Incremental pre-solve local'] = '#ff7f0e'
#    '#7f7f7f',  // middle gray
#    '#bcbd22',  // curry yellow-green
#    '#17becf'   // blue-teal
def plot_running_times(names,csvs,sortby,normalize=-1,plottitle="", orderFile="",yaxisrange=None,directory="",showtable=False,showfig=True):
    T = pd.read_csv(csvs[0], header=None)
    T[1] = T[1] / 1000000
    for i in range(1,len(names)):
        F2 = pd.read_csv(csvs[i],header=None)
        T[i+1] = F2[1] / 1000000
    #T = T.sort_values(by=[sortby+1])
    if orderFile != "":
        F = pd.read_csv(orderFile, header=None)
        T[len(names)+2] = F[1]
        T = T.sort_values(by=[len(names)+2])

    X=T[0].tolist()

    for i in range(len(X)):
        X[i] = 'Game ' + str(X[i])
    
    table="\\begin{tabular}{|"
    for i in range(len(names)+1):
        table += 'l|'
    table += "}\n\hline\n "
    for n in names:
        table += '& ' + n + ' '
    table += "\\\\ \hline\n"
    for y in range(len(X)):
        table += str(X[y]) + " "
        for x in range(len(names)):
            table += '& '
            val = str(T[x+1].tolist()[y])
            if val == 'nan':
                val = '\\dag'
            else:
                val += ' ms'
            table += val
            table += ' '
        table += "\\\\ \hline\n"
    table += "\\end{tabular}"
    
    if normalize > 0:
        for i in range(1 , len(names)+1):
            if i != normalize:
                T[i] = T[i] / T[normalize]
                #for j in range(0,len(T[i])):
                   # if T[i][j] < 1:
                    #    T[i][j] = -1 / T[i][j]
        T[normalize] = T[normalize] / T[normalize]
    
    if normalize > 0:
        D = [None] * (len(names))
        D[0] = go.Scatter(name=names[normalize-1], x=X, y=T[normalize].tolist(),
                          line=dict(color=colors[names[normalize-1]]))
        c = 1
    else:
        D = [None] * len(names)
        c = 0
    for i in range(len(names)):
        if names[i] in colors:
            colorite = [colors[names[i]]] * len(X)
            for j in range(len(X)):
                if math.isnan(T[i+1][j]):
                    T[i+1][j] = T[1][j] * 2
                    colorite[j] = 'lightslategray'
            if i != normalize - 1:
                D[c] = go.Bar(name=names[i], x=X, y=T[i+1].tolist(),marker_color=colorite)
                c = c + 1
        else:
            print "Unknown name " + names[i]
    
    fig = go.Figure(data=D)
    # Change the bar mode
    fig.update_layout(
         legend=go.layout.Legend(
            x=0.7,
            y=1,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=12,
                color="black"
            ),
            bgcolor="LightSteelBlue",
            bordercolor="Black",
            borderwidth=2
        ),
        yaxis_type='log',
        barmode='group',
#         title=go.layout.Title(
#             text=plottitle
#         ),
    )
    if showtable:
        print table
    if yaxisrange != None:
        fig.update_yaxes(range=yaxisrange)
    if directory != "":
        filename = ""
        for n in names:
            filename += n + "_"
        filename = directory + "/" + filename + ".png"
        fig.write_image(filename,width=1000)
    if showfig:
        fig.show()

In [26]:
# Fam compared to Zlnk product
for p in ["minepump","elevator"]:
    print p
    plot_running_times(
        ['Zlnk product based','Zlnk local product based'],
        [
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk_local.data.csv"
        ],0
        ,directory="/home/sjef/AfstudeerRepo/Report/results/"+p
        )
for p in ["FF_randomgames","FC_randomgames","BC_randomgames","randomscalegames"]:
    print p
    plot_running_times(
        ['Zlnk product based','Zlnk local product based'],
        [
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk_local.data.csv"
        ],0,normalize=1
        ,directory="/home/sjef/AfstudeerRepo/Report/results/"+p
        )

minepump


elevator


FF_randomgames


FC_randomgames


BC_randomgames


randomscalegames


In [3]:
for p in ["minepump","elevator","FF_randomgames","FC_randomgames","BC_randomgames","randomscalegames"]:
    print "\subsection{"+p.replace('_',' ')+"}"
    plot_running_times(['Zlnk product based'],["/home/sjef/AfstudeerRepo/implementation/testsuite/data/singles/csvs/clean_"+p+"_zlnk.data.csv"],0,showfig=False,showtable=True)
    plot_running_times(['Fixed-point product based'],["/home/sjef/AfstudeerRepo/implementation/testsuite/data/singles/csvs/clean_"+p+"_MBR.data.csv"],0,showfig=False,showtable=True)
    plot_running_times(['Fixed-point local product based'],["/home/sjef/AfstudeerRepo/implementation/testsuite/data/singles/csvs/clean_"+p+"_MBR_local.data.csv"],0,showfig=False,showtable=True)
    
    plot_running_times(['Zlnk fam based - explicit'],["/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk_explicit.data.csv"],0,showfig=False,showtable=True)
    plot_running_times(['Zlnk fam based - symbolic'],["/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk.data.csv"],0,showfig=False,showtable=True)
    
    plot_running_times(['Incremental pre-solve'],["/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_MBR.data.csv"],0,showfig=False,showtable=True)
    plot_running_times(['Incremental pre-solve local'],["/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_MBR_local.data.csv"],0,showfig=False,showtable=True)
    
    

\subsection{minepump}
\begin{tabular}{|l|l|}
\hline
 & Zlnk product based \\ \hline
Game 1 & 34.083326 ms \\ \hline
Game 2 & 56.145233 ms \\ \hline
Game 3 & 174.507256 ms \\ \hline
Game 4 & 132.617902 ms \\ \hline
Game 5 & 144.362213 ms \\ \hline
Game 6 & 223.132309 ms \\ \hline
Game 7 & 126.891372 ms \\ \hline
Game 8 & 19.067614 ms \\ \hline
Game 9 & 106.66577 ms \\ \hline
\end{tabular}
\begin{tabular}{|l|l|}
\hline
 & Fixed-point product based \\ \hline
Game 1 & 67.932874 ms \\ \hline
Game 2 & 83.635694 ms \\ \hline
Game 3 & 289.122412 ms \\ \hline
Game 4 & 37776.832166 ms \\ \hline
Game 5 & 1199.767639 ms \\ \hline
Game 6 & 745.95261 ms \\ \hline
Game 7 & 566.40671 ms \\ \hline
Game 8 & 8.47649 ms \\ \hline
Game 9 & 720.249898 ms \\ \hline
\end{tabular}
\begin{tabular}{|l|l|}
\hline
 & Fixed-point local product based \\ \hline
Game 1 & 67.901337 ms \\ \hline
Game 2 & 84.091102 ms \\ \hline
Game 3 & 291.25624 ms \\ \hline
Game 4 & 37794.125674 ms \\ \hline
Game 5 & 836.490805 ms \\ \

\begin{tabular}{|l|l|}
\hline
 & Fixed-point local product based \\ \hline
Game 50 & 20.715406 ms \\ \hline
Game 51 & 6288.423101 ms \\ \hline
Game 52 & 1.408566 ms \\ \hline
Game 53 & 4.299501 ms \\ \hline
Game 54 & 29.69588 ms \\ \hline
Game 55 & 7.457633 ms \\ \hline
Game 56 & 480.916207 ms \\ \hline
Game 57 & 169.365719 ms \\ \hline
Game 58 & 6.796605 ms \\ \hline
Game 59 & 10.329544 ms \\ \hline
Game 60 & 267.473609 ms \\ \hline
Game 61 & 1.490853 ms \\ \hline
Game 62 & 206.409492 ms \\ \hline
Game 63 & 136.795202 ms \\ \hline
Game 64 & 29.241653 ms \\ \hline
Game 65 & 203.789439 ms \\ \hline
Game 66 & 120.934348 ms \\ \hline
Game 67 & 20.884496 ms \\ \hline
Game 68 & 19.090302 ms \\ \hline
Game 69 & 28.122392 ms \\ \hline
Game 70 & 0.391556 ms \\ \hline
Game 71 & 27.125678 ms \\ \hline
Game 72 & 0.213712 ms \\ \hline
Game 73 & 3.473444 ms \\ \hline
Game 74 & 143.809764 ms \\ \hline
Game 75 & 37.416916 ms \\ \hline
Game 76 & 170.780492 ms \\ \hline
Game 77 & 385.720314 ms \\ \hlin

\begin{tabular}{|l|l|}
\hline
 & Zlnk product based \\ \hline
Game 50 & 23.712499 ms \\ \hline
Game 51 & 242.041841 ms \\ \hline
Game 52 & 3.4723 ms \\ \hline
Game 53 & 2.714006 ms \\ \hline
Game 54 & 48.719568 ms \\ \hline
Game 55 & 3.155165 ms \\ \hline
Game 56 & 204.548397 ms \\ \hline
Game 57 & 9.29805 ms \\ \hline
Game 58 & 2.715256 ms \\ \hline
Game 59 & 12.915748 ms \\ \hline
Game 60 & 86.027465 ms \\ \hline
Game 61 & 2.68495 ms \\ \hline
Game 62 & 77.716429 ms \\ \hline
Game 63 & 20.043507 ms \\ \hline
Game 64 & 10.663885 ms \\ \hline
Game 65 & 155.866176 ms \\ \hline
Game 66 & 18.884118 ms \\ \hline
Game 67 & 3.083076 ms \\ \hline
Game 68 & 47.288989 ms \\ \hline
Game 69 & 23.226832 ms \\ \hline
Game 70 & 1.693863 ms \\ \hline
Game 71 & 34.141341 ms \\ \hline
Game 72 & 1.189907 ms \\ \hline
Game 73 & 2.908208 ms \\ \hline
Game 74 & 38.724408 ms \\ \hline
Game 75 & 111.915312 ms \\ \hline
Game 76 & 194.080224 ms \\ \hline
Game 77 & 43.542376 ms \\ \hline
Game 78 & 159.817194 ms

\begin{tabular}{|l|l|}
\hline
 & Incremental pre-solve local \\ \hline
Game 50 & 270.824584 ms \\ \hline
Game 51 & 61392.839784 ms \\ \hline
Game 52 & 4.284938 ms \\ \hline
Game 53 & 14.20193 ms \\ \hline
Game 54 & 7935.80916 ms \\ \hline
Game 55 & 56.676909 ms \\ \hline
Game 56 & 232.684629 ms \\ \hline
Game 57 & 638.597981 ms \\ \hline
Game 58 & 21.033317 ms \\ \hline
Game 59 & 184.539532 ms \\ \hline
Game 60 & 2397.387574 ms \\ \hline
Game 61 & 1.685734 ms \\ \hline
Game 62 & 1125.623772 ms \\ \hline
Game 63 & 679.047522 ms \\ \hline
Game 64 & 2.873935 ms \\ \hline
Game 65 & 160.100314 ms \\ \hline
Game 66 & 123.322917 ms \\ \hline
Game 67 & 11.267302 ms \\ \hline
Game 68 & 14.983563 ms \\ \hline
Game 69 & 461.775982 ms \\ \hline
Game 70 & 5.687645 ms \\ \hline
Game 71 & 4.391004 ms \\ \hline
Game 72 & 1.560376 ms \\ \hline
Game 73 & 29.228599 ms \\ \hline
Game 74 & 3.844283 ms \\ \hline
Game 75 & 81.336876 ms \\ \hline
Game 76 & 136.94918 ms \\ \hline
Game 77 & 15.475165 ms \\ \hli

\begin{tabular}{|l|l|}
\hline
 & Zlnk fam based - symbolic \\ \hline
Game 50 & 132.021157 ms \\ \hline
Game 51 & 38845.561463 ms \\ \hline
Game 52 & 3.276376 ms \\ \hline
Game 53 & 6.825309 ms \\ \hline
Game 54 & 1792.450705 ms \\ \hline
Game 55 & 77.522421 ms \\ \hline
Game 56 & 15689.82599 ms \\ \hline
Game 57 & 57.049405 ms \\ \hline
Game 58 & 45.981337 ms \\ \hline
Game 59 & 69.735921 ms \\ \hline
Game 60 & 1668.572348 ms \\ \hline
Game 61 & 44.427574 ms \\ \hline
Game 62 & 839.471905 ms \\ \hline
Game 63 & 704.368702 ms \\ \hline
Game 64 & 59.404315 ms \\ \hline
Game 65 & 13004.347618 ms \\ \hline
Game 66 & 95.554262 ms \\ \hline
Game 67 & 55.123488 ms \\ \hline
Game 68 & 384.668654 ms \\ \hline
Game 69 & 87.202018 ms \\ \hline
Game 70 & 0.33217 ms \\ \hline
Game 71 & 278.714502 ms \\ \hline
Game 72 & 0.994597 ms \\ \hline
Game 73 & 14.938285 ms \\ \hline
Game 74 & 74.911809 ms \\ \hline
Game 75 & 9.660759 ms \\ \hline
Game 76 & 11418.509168 ms \\ \hline
Game 77 & 417.293073 ms \\

\begin{tabular}{|l|l|}
\hline
 & Fixed-point local product based \\ \hline
Game 2a & 0.274736 ms \\ \hline
Game 2b & 0.486742 ms \\ \hline
Game 2c & 3.825711 ms \\ \hline
Game 2d & 4.232471 ms \\ \hline
Game 3c & 39.746109 ms \\ \hline
Game 3d & 16.270579 ms \\ \hline
Game 3 & 2.419012 ms \\ \hline
Game 4a & 2.783131 ms \\ \hline
Game 4b & 0.252504 ms \\ \hline
Game 4c & 0.132024 ms \\ \hline
Game 4d & 5.692369 ms \\ \hline
Game 4 & 0.453256 ms \\ \hline
Game 5a & 1.72395 ms \\ \hline
Game 5b & 3.83336 ms \\ \hline
Game 5c & 0.534967 ms \\ \hline
Game 5d & 11.636036 ms \\ \hline
Game 6a & 5.352781 ms \\ \hline
Game 6b & 13.134473 ms \\ \hline
Game 6c & 196.969686 ms \\ \hline
Game 6d & 37.773285 ms \\ \hline
Game 7a & 66.9029 ms \\ \hline
Game 7b & 18.025315 ms \\ \hline
Game 7c & 3.233681 ms \\ \hline
Game 7d & 12.348907 ms \\ \hline
Game 8a & 44.547464 ms \\ \hline
Game 8b & 22.910169 ms \\ \hline
Game 8c & 49.395582 ms \\ \hline
Game 8d & 55.361792 ms \\ \hline
Game 9a & 126.744039 

In [27]:
# Fam compared to Zlnk product
for p in ["minepump","elevator"]:
    print p
    plot_running_times(
        ['Zlnk product based','Zlnk fam based - explicit','Zlnk fam based - symbolic'],
        [
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/singles/csvs/clean_"+p+"_zlnk.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk_explicit.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk.data.csv"
        ],0
        ,directory="/home/sjef/AfstudeerRepo/Report/results/"+p
        )
for p in ["FF_randomgames","FC_randomgames","BC_randomgames","randomscalegames"]:
    print p
    plot_running_times(
        ['Zlnk product based','Zlnk fam based - explicit','Zlnk fam based - symbolic'],
        [
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/singles/csvs/clean_"+p+"_zlnk.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk_explicit.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk.data.csv"
        ],0,normalize=1
        ,directory="/home/sjef/AfstudeerRepo/Report/results/"+p
        )

minepump


elevator


FF_randomgames


FC_randomgames


BC_randomgames


randomscalegames


In [24]:
# Fam compared to Zlnk product
for p in ["minepump","elevator"]:
    print p
    plot_running_times(
        ['Zlnk local product based','Zlnk fam based - explicit local','Zlnk fam based - symbolic local'],
        [
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/singles/csvs/clean_"+p+"_zlnk_local.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk_explicit_local.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk_local.data.csv"
        ],0
        ,directory="/home/sjef/AfstudeerRepo/Report/results/"+p
        )
for p in ["FF_randomgames","FC_randomgames","BC_randomgames","randomscalegames"]:
    print p
    plot_running_times(
        ['Zlnk local product based','Zlnk fam based - explicit local','Zlnk fam based - symbolic local'],
        [
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/singles/csvs/clean_"+p+"_zlnk_local.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk_explicit_local.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk_local.data.csv"
        ],0,normalize=1
        ,directory="/home/sjef/AfstudeerRepo/Report/results/"+p
        )

minepump


elevator


FF_randomgames


FC_randomgames


BC_randomgames


randomscalegames


In [5]:
# Fam compared to Zlnk product
for p in ["minepump","elevator"]:
    print p
    plot_running_times(
        ['Zlnk product based','Incremental pre-solve','Incremental pre-solve local'],
        [
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/singles/csvs/clean_"+p+"_zlnk.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_MBR.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_MBR_local.data.csv"
        ],0
        ,directory="/home/sjef/AfstudeerRepo/Report/results/"+p
        )
for p in ["FF_randomgames","FC_randomgames","BC_randomgames", "randomscalegames"]:
    print p
    plot_running_times(
        ['Zlnk product based','Incremental pre-solve','Incremental pre-solve local'],
        [
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/singles/csvs/clean_"+p+"_zlnk.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_MBR.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_MBR_local.data.csv"
        ],0,normalize=1
        ,directory="/home/sjef/AfstudeerRepo/Report/results/"+p
        )

minepump


elevator


FF_randomgames


FC_randomgames


BC_randomgames


randomscalegames


In [6]:
# FPite compared to inc
for p in ["minepump","elevator"]:
    print p
    plot_running_times(
        ['Fixed-point product based','Incremental pre-solve'],
        [
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/singles/csvs/clean_"+p+"_MBR.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_MBR.data.csv"
        ],0
        ,directory="/home/sjef/AfstudeerRepo/Report/results/"+p
        )
for p in ["FF_randomgames","FC_randomgames","BC_randomgames", "randomscalegames"]:
    print p
    plot_running_times(
        ['Fixed-point product based','Incremental pre-solve'],
        [
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/singles/csvs/clean_"+p+"_MBR.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_MBR.data.csv"
        ],0,normalize=1
        ,directory="/home/sjef/AfstudeerRepo/Report/results/"+p
        )

minepump


elevator


FF_randomgames


FC_randomgames


BC_randomgames


randomscalegames


In [7]:
# FPite compared to inc
for p in ["minepump","elevator"]:
    print p
    plot_running_times(
        ['Fixed-point local product based','Incremental pre-solve local'],
        [
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/singles/csvs/clean_"+p+"_MBR_local.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_MBR_local.data.csv"
        ],0
        ,directory="/home/sjef/AfstudeerRepo/Report/results/"+p
        )
for p in ["FF_randomgames","FC_randomgames","BC_randomgames","randomscalegames"]:
    print p
    plot_running_times(
        ['Fixed-point local product based','Incremental pre-solve local'],
        [
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/singles/csvs/clean_"+p+"_MBR_local.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_MBR_local.data.csv"
        ],0,normalize=1
        ,directory="/home/sjef/AfstudeerRepo/Report/results/"+p
        )

minepump


elevator


FF_randomgames


FC_randomgames


BC_randomgames


randomscalegames


In [8]:
# FPite compared to inc
for p in ["minepump","elevator"]:
    print p
    plot_running_times(
        ['Zlnk fam based - symbolic','Incremental pre-solve local'],
        [
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_MBR_local.data.csv"
        ],0
        ,directory="/home/sjef/AfstudeerRepo/Report/results/"+p
        )
for p in ["FF_randomgames","FC_randomgames","BC_randomgames","randomscalegames"]:
    print p
    plot_running_times(
        ['Zlnk fam based - symbolic','Incremental pre-solve local'],
        [
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_zlnk.data.csv",
            "/home/sjef/AfstudeerRepo/implementation/testsuite/data/family/csvs/clean_"+p+"_MBR_local.data.csv"
        ],0,normalize=1

        ,directory="/home/sjef/AfstudeerRepo/Report/results/"+p
        )

minepump


elevator


FF_randomgames


FC_randomgames


BC_randomgames


randomscalegames


In [9]:
def printtable(algnames, algfiles, algdir):
    table="\\begin{tabular}{|"
    for i in range(len(algnames)+1):
        table += 'l|'
    table += "}\n\hline\n "
    for n in algnames:
        table += '& ' + n + ' '
    table += "\\\\ \hline\n"

    for problem in ['minepump','elevator','FF_randomgames','FC_randomgames','BC_randomgames','randomscalegames']:
        table += problem.replace('_',' ')
        for a in range(len(algnames)):
            c="/home/sjef/AfstudeerRepo/implementation/testsuite/data/"+algdir[a]+"/csvs/clean_"+problem+"_"+algfiles[a]+".data.csv"
            T = pd.read_csv(c, header=None)
            T[1] = T[1] / 1000000
            table += '& ' + str(int(round(T[1].mean()))) + ' ms'
        table += '\\\\ \hline\n'
    table += "\\end{tabular}"
    print table
printtable(
    ['Zlnk product based',
                     'Fixed-point product based',
                     'Fixed-point local product based'],
    ['zlnk','MBR','MBR_local'],
     ['singles','singles','singles'])
printtable(
    [
                     'Zlnk fam based - explicit',
                     'Zlnk fam based - symbolic',
                     'Incremental pre-solve',
                     'Incremental pre-solve local'],
    ['zlnk_explicit','zlnk','MBR','MBR_local'],
    ['family','family','family','family'])

\begin{tabular}{|l|l|l|l|}
\hline
 & Zlnk product based & Fixed-point product based & Fixed-point local product based \\ \hline
minepump& 113 ms& 4606 ms& 4474 ms\\ \hline
elevator& 35062 ms& 9733571 ms& 1707949 ms\\ \hline
FF randomgames& 72 ms& 418 ms& 327 ms\\ \hline
FC randomgames& 69 ms& 418 ms& 386 ms\\ \hline
BC randomgames& 70 ms& 422 ms& 356 ms\\ \hline
randomscalegames& 134 ms& 423 ms& 395 ms\\ \hline
\end{tabular}
\begin{tabular}{|l|l|l|l|l|}
\hline
 & Zlnk fam based - explicit & Zlnk fam based - symbolic & Incremental pre-solve & Incremental pre-solve local \\ \hline
minepump& 112 ms& 16 ms& 1153 ms& 671 ms\\ \hline
elevator& 105856 ms& 6674 ms& 1065377 ms& 378460 ms\\ \hline
FF randomgames& 29 ms& 60 ms& 276 ms& 40 ms\\ \hline
FC randomgames& 36 ms& 2675 ms& 8146 ms& 1528 ms\\ \hline
BC randomgames& 35 ms& 2191 ms& 12506 ms& 4395 ms\\ \hline
randomscalegames& 16 ms& 9 ms& 206 ms& 24 ms\\ \hline
\end{tabular}
